# Description #

This notebook is used for the level 3 integration tests from test plan LVV-P81 (https://jira.lsstcorp.org/secure/Tests.jspa#/testPlan/LVV-P81) as part of test cylce LVV-C176 (https://jira.lsstcorp.org/secure/Tests.jspa#/testCycle/LVV-C176). The following tests are currently run as part of this notebook:

 - LVV-T2193 (https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/LVV-T2193)
 
Upon completion, save the notebook and its output as a pdf file to be attached to the test execution in JIRA. 

Last executed by E. Dennihy 20210928

## Setup ##

This setup is applicable to all three of the tests (LVV-T2190, LVV-T2193, LVV-T2290). 

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import rubin_jupyter_utils.lab.notebook as nb
nb.utils.get_node()

Patching auth into notebook.base.handlers.IPythonHandler(notebook.base.handlers.AuthenticatedHandler) -> IPythonHandler(jupyterhub.singleuser.mixins.HubAuthenticatedHandler, notebook.base.handlers.AuthenticatedHandler)


'andes03.cp.lsst.org'

In [3]:
import os
import sys
import asyncio
import logging

import pandas as pd
import numpy as np

from matplotlib import pyplot as plt

from lsst.ts import salobj
from lsst.ts.observatory.control.maintel import MTCS, ComCam
from lsst.ts.observatory.control import RotType

Update leap second table
current_tai uses the system TAI clock


In [4]:
logging.basicConfig(format="%(name)s:%(message)s", level=logging.DEBUG)

In [5]:
log = logging.getLogger("setup")
log.level = logging.DEBUG

In [6]:
domain = salobj.Domain()

In [7]:
mtcs = MTCS(domain=domain, log=log)
mtcs.set_rem_loglevel(40)

mtmount: Adding all resources.
mtptg: Adding all resources.
mtaos: Adding all resources.
mtm1m3: Adding all resources.
mtm2: Adding all resources.
mthexapod_1: Adding all resources.
mthexapod_2: Adding all resources.
mtrotator: Adding all resources.
mtdome: Adding all resources.
mtdometrajectory: Adding all resources.
Read historical data in 0.04 sec
Read historical data in 0.09 sec
elevation DDS read queue is full (100 elements); data may be lost
motors DDS read queue is full (100 elements); data may be lost
electrical DDS read queue is full (100 elements); data may be lost
cameraCableWrap DDS read queue is full (100 elements); data may be lost
electrical DDS read queue is full (100 elements); data may be lost
application DDS read queue is full (100 elements); data may be lost
azimuth DDS read queue is full (100 elements); data may be lost
ccwFollowingError DDS read queue is full (100 elements); data may be lost
actuators DDS read queue is full (100 elements); data may be lost
powerSu

In [8]:
await mtcs.start_task

[None, None, None, None, None, None, None, None, None, None]

In [9]:
comcam = ComCam(domain=domain, log=log)

cccamera: Adding all resources.
ccheaderservice: Adding all resources.
ccarchiver: Adding all resources.
Read historical data in 0.01 sec
Read historical data in 0.08 sec
Read historical data in 0.24 sec


In [10]:
comcam.set_rem_loglevel(40)

In [11]:
await comcam.start_task

[None, None, None]

In [12]:
await comcam.enable()

Enabling all components
Gathering settings.
Couldn't get settingVersions event. Using empty settings.
Couldn't get settingVersions event. Using empty settings.
Complete settings for cccamera.
Complete settings for ccheaderservice.
Complete settings for ccarchiver.
Settings versions: {'cccamera': '', 'ccheaderservice': '', 'ccarchiver': ''}
[cccamera]::[<State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]
[ccheaderservice]::[<State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]
[ccarchiver]::[<State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]
All components in <State.ENABLED: 2>.


## LVV-T2290 ##

Here we perform a test to slew and take an image

Test LVV-T2290 Execution LVV-E1349

In [13]:
target = await mtcs.find_target(el=60, az=120, mag_limit=8)
print(target)

HD 116969


In [14]:
await mtcs.slew_object(target, rot_type=RotType.PhysicalSky, rot=1.9)

Slewing to HD 116969: 13 27 51.5573 -40 23 38.494
Setting rotator physical position to 1.9 deg. Rotator will track sky.
Stop tracking.
Sending slew command.
Scheduling check coroutines
process as completed...
Monitor position started.
Waiting for Target event from mtmount.
mtmount: <State.ENABLED: 2>
mtptg: <State.ENABLED: 2>
mtaos: <State.ENABLED: 2>
mtm1m3: <State.ENABLED: 2>
mtm2: <State.ENABLED: 2>
mthexapod_1: <State.ENABLED: 2>
mthexapod_2: <State.ENABLED: 2>
mtrotator: <State.ENABLED: 2>
mtdome: <State.ENABLED: 2>
mtdometrajectory: <State.ENABLED: 2>
Wait for mtmount in position event.
Wait for dome in position event.
Wait for rotator in position event.
MTMount axesInPosition got: elevation True, azimuth False.
MTMount axesInPosition got: elevation False, azimuth False.
Mount target: private_revCode: 261ad639, private_sndStamp: 1632842498.3596807, private_rcvStamp: 1632842498.36308, private_seqNum: 2, private_identity: MTMount, private_origin: 221230, private_host: 0, elevation:

telescope is in position, let's take a comcam image!

In [15]:
await comcam.take_object(15)

Generating group_id
imagetype: OBJECT, TCS synchronization not configured.
OBJECT 0001 - 0001


array([2021092800001])

In [16]:
await mtcs.stop_tracking()

Stop tracking.


The image landed correctly and even has some correct info in the file header, later we will need to do a more careful analysis of the header information

## LVV-T2190 ##

Now we are moving to on mtaos testing 

Test LVV-T2190 Execution LVV-1351

initialize an array of wavefront error values to be passed directly to the mtaos

In [17]:
import numpy as np
wavefront_errors = np.zeros(19)

In [18]:
wavefront_errors[3] += 0.1 # add 0.1 um to z7

In [19]:
await mtcs.rem.mtaos.cmd_addAberration.set_start(wf=wavefront_errors, timeout=10)

This command primes the corrections, the issueCorrection command is needed to actually command them to be sent

In [20]:
await mtcs.rem.mtaos.cmd_issueCorrection.start(timeout=60.)

In [21]:
await mtcs.rem.mtaos.cmd_resetCorrection.start()

In [23]:
wavefront_errors[3] = 0.2 # add 1.0 um of z7

In [24]:
await mtcs.rem.mtaos.cmd_addAberration.set_start(wf=wavefront_errors, timeout=10)

In [25]:
await mtcs.rem.mtaos.cmd_issueCorrection.start(timeout=60.)

In [ ]:
wavefront_errors[3] = 1.0 # add 1.0 um of z7

In [ ]:
await mtcs.rem.mtaos.cmd_addAberration.set_start(wf=wavefront_errors, timeout=10)

In [ ]:
await mtcs.rem.mtaos.cmd_issueCorrection.start(timeout=60.)

## LVV-T2193 ##

Moving to a failure test of how the mtaos handles a rejected command

JIRA Test LVV-T2193 Execution LVV-E1400

In [26]:
await mtcs.rem.mtaos.cmd_resetCorrection.start()

In [27]:
await mtcs.rem.mtaos.cmd_issueCorrection.start(timeout=60.)

In [28]:
wavefront_errors

array([0. , 0. , 0. , 0.2, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. ])

In [30]:
wavefront_errors[3]=1.0

In [31]:
await mtcs.rem.mtaos.cmd_addAberration.set_start(wf=wavefront_errors, timeout=10)

In [32]:
await mtcs.rem.mtaos.cmd_issueCorrection.start(timeout=60.)

In [33]:
await mtcs.set_state(salobj.State.DISABLED, components=["mthexapod_2"])

[mthexapod_2]::[<State.ENABLED: 2>, <State.DISABLED: 1>]
All components in <State.DISABLED: 1>.


In [34]:
await mtcs.rem.mtaos.cmd_addAberration.set_start(wf=wavefront_errors, timeout=10)

In [35]:
await mtcs.rem.mtaos.cmd_issueCorrection.start(timeout=60.)

AckError: msg='Command failed', ackcmd=(ackcmd private_seqNum=763213323, ack=<SalRetCode.CMD_FAILED: -302>, error=1, result="Failed: Failed to apply correction to: ['m2hex']. ")

In [36]:
await mtcs.set_state(salobj.State.ENABLED, components=["mthexapod_2"])

[mthexapod_2]::[<State.DISABLED: 1>, <State.ENABLED: 2>]
All components in <State.ENABLED: 2>.


In [37]:
await mtcs.rem.mtaos.cmd_issueCorrection.start(timeout=60.)

In [38]:
await mtcs.rem.mtaos.cmd_addAberration.set_start(wf=wavefront_errors, timeout=10)

In [39]:
await mtcs.rem.mtaos.cmd_issueCorrection.start(timeout=60.)

In [40]:
await mtcs.rem.mtaos.cmd_rejectCorrection.start()

In [41]:
await mtcs.rem.mtaos.cmd_issueCorrection.start(timeout=60.)

In [42]:
wavefront_errors[3] += 1.0

In [43]:
wavefront_errors

array([0., 0., 0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.])

In [44]:
await mtcs.rem.mtaos.cmd_addAberration.set_start(wf=wavefront_errors, timeout=10)

In [45]:
await mtcs.rem.mtaos.cmd_issueCorrection.start(timeout=60.)

## Wrap Up and Shut Down ##

In [46]:
await mtcs.set_state(salobj.State.STANDBY, components=["mtaos"])

[mtaos]::[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]
All components in <State.STANDBY: 5>.


In [47]:
await mtcs.lower_m1m3()

M1M3 current detailed state {<DetailedState.ACTIVEENGINEERING: 11>, <DetailedState.ACTIVE: 7>}, executing command...
process as completed...
M1M3 detailed state 8
mtm1m3: <State.ENABLED: 2>
mtm1m3: <State.ENABLED: 2>
M1M3 detailed state 5


In [48]:
await mtcs.set_state(salobj.State.STANDBY, components=["mtm1m3"])

[mtm1m3]::[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]
All components in <State.STANDBY: 5>.


In [50]:
await mtcs.set_state(salobj.State.STANDBY, components=["mtm2"])

[mtm2]::[<State.DISABLED: 1>, <State.STANDBY: 5>]
All components in <State.STANDBY: 5>.


In [51]:
await mtcs.set_state(salobj.State.STANDBY, components=["mthexapod_1"])

[mthexapod_1]::[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]
All components in <State.STANDBY: 5>.


In [52]:
await mtcs.set_state(salobj.State.STANDBY, components=["mthexapod_2"])

[mthexapod_2]::[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]
All components in <State.STANDBY: 5>.


In [53]:
await mtcs.standby()

[mtmount]::[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]
[mtptg]::[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]
[mtaos]::[<State.STANDBY: 5>]
[mtm1m3]::[<State.STANDBY: 5>]
[mtm2]::[<State.STANDBY: 5>]
[mthexapod_1]::[<State.STANDBY: 5>]
[mthexapod_2]::[<State.STANDBY: 5>]
[mtrotator]::[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]
[mtdome]::[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]
[mtdometrajectory]::[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]
All components in <State.STANDBY: 5>.


In [54]:
await comcam.standby()

[cccamera]::[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]
[ccheaderservice]::[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]
[ccarchiver]::[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]
All components in <State.STANDBY: 5>.


## Appendix ##

In [ ]:
# Setting MTMtg to STANDBY so Tiago can change the slew limits which should allows
#  us moving to rot=-1.5
await mtcs.set_state(salobj.State.STANDBY, components=["mtptg"])

In [ ]:
await mtcs.set_state(salobj.State.ENABLED, components=["mtptg"])

In [ ]:
await mtcs.slew_object(target, rot_type=RotType.PhysicalSky, rot=-1.5)

In [ ]:
await mtcs.stop_tracking()

In [ ]:
await mtcs.set_state(
    state=salobj.State.ENABLED,
    settings=dict(mthexapod_1="default"),
    components=["mthexapod_1"]
    )

In [ ]:
await mtcs.rem.mthexapod_1.cmd_clearError.set_start()

In [ ]:
await mtcs.set_state(salobj.State.ENABLED, components=["mtptg"])

In [ ]:
await mtcs.set_state(
    state=salobj.State.ENABLED,
    settings=dict(mthexapod_1="default"),
    components=["mthexapod_1"]
    )

In [ ]:
target = await mtcs.find_target(el=60, az=120, mag_limit=8)
print(target)

In [ ]:
await mtcs.slew_object(target, rot_type=RotType.PhysicalSky)

In [ ]:
# await mtcs.stop_tracking()

In [ ]:
await mtcs.rem.mtrotator.cmd_clearError.set_start()

In [ ]:
await mtcs.set_state(salobj.State.ENABLED, components=["mtrotator"])

In [ ]:
await mtcs.set_state(salobj.State.ENABLED, components=["mtptg"])

In [ ]:
await mtcs.slew_object(target, rot_type=RotType.PhysicalSky)

In [ ]:
await mtcs.stop_tracking()

In [ ]:
await mtcs.slew_object(target, rot_type=RotType.PhysicalSky, rot=1.9)

In [ ]:
await mtcs.rem.mtrotator.cmd_clearError.set_start()

In [ ]:
await mtcs.set_state(salobj.State.ENABLED, components=["mtrotator"])

In [ ]:
await mtcs.set_state(salobj.State.ENABLED, components=["mtptg"])

In [ ]:
await mtcs.slew_object(target, rot_type=RotType.PhysicalSky, rot=1.5)

In [ ]:
await comcam.take_object(1)

In [ ]:
await mtcs.stop_tracking()

In [ ]:
await mtcs.slew_object(target, rot_type=RotType.Physical, rot=0)

In [ ]:
await mtcs.rem.mtrotator.cmd_clearError.set_start()
await mtcs.set_state(salobj.State.ENABLED, components=["mtrotator"])

In [ ]:
await mtcs.set_state(salobj.State.ENABLED, components=["mtptg"])

In [ ]:
await mtcs.slew_object(target, rot_type=RotType.Physical, rot=0)

In [ ]:
await comcam.get_available_filters()

In [ ]:
await comcam.setup_filter('r_03')

In [ ]:
await mtcs.rem.mthexapod_1.cmd_clearError.set_start()

In [ ]:
await mtcs.set_state(
    state=salobj.State.ENABLED,
    settings=dict(mthexapod_1="default"),
    components=["mthexapod_1"]
    )

In [ ]:
await mtcs.slew_object(target, rot_type=RotType.PhysicalSky, rot=1.5)

In [ ]:
await mtcs.rem.mtrotator.cmd_clearError.set_start()

In [ ]:
await mtcs.set_state(salobj.State.ENABLED, components=["mtrotator"])

In [ ]:
await mtcs.set_state(salobj.State.ENABLED, components=["mtptg"])

In [ ]:
await mtcs.slew_object(target, rot_type=RotType.PhysicalSky, rot=1.5)

In [ ]:
await comcam.take_object(15)

In [ ]:
await mtcs.stop_tracking()

In [ ]:
await comcam.standby()